In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
x_train = x_train / 255
x_test = x_test / 255
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [4]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [5]:
print(y_train.shape, y_test.shape)

(60000,) (10000,)


In [6]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)
print(y_train.shape, y_test.shape)

(60000, 10) (10000, 10)


In [7]:
learning_rate = 0.001
training_epochs = 12
batch_size = 128

In [8]:
def get_model():
    X = tf.keras.layers.Input(shape=[28, 28, 1])
    
    H = tf.keras.layers.Conv2D(filters=16, kernel_size=3)(X)
    H = tf.keras.layers.Activation('relu')(H)
    H = tf.keras.layers.MaxPool2D(pool_size=2)(H)

    H = tf.keras.layers.Conv2D(filters=32, kernel_size=3)(H)
    H = tf.keras.layers.Activation('relu')(H)
    H = tf.keras.layers.MaxPool2D(pool_size=2)(H)

    H = tf.keras.layers.Flatten()(H)
    
    H = tf.keras.layers.Dense(100, kernel_initializer='glorot_normal')(H)
    H = tf.keras.layers.BatchNormalization()(H)
    H = tf.keras.layers.Activation('relu')(H)

    Y = tf.keras.layers.Dense(10, kernel_initializer='glorot_normal',
                         activation='softmax')(H)

    return tf.keras.models.Model(X, Y)

'''
model.compile(loss='categorical_crossentropy', 
             optimizer = tf.keras.optimizers.Adam(lr=learning_rate),
             metrics='accuracy')
model.summary()
'''

"\nmodel.compile(loss='categorical_crossentropy', \n             optimizer = tf.keras.optimizers.Adam(lr=learning_rate),\n             metrics='accuracy')\nmodel.summary()\n"

In [9]:
model1 = get_model()
model2 = get_model()
model3 = get_model()
model4 = get_model()
model5 = get_model()

In [13]:
inputs = tf.keras.layers.Input(shape=[28, 28, 1])
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)
y4 = model4(inputs)
y5 = model5(inputs)
outputs = tf.keras.layers.average([y1, y2, y3, y4, y5])
ensemble_model = tf.keras.models.Model(inputs, outputs)
ensemble_model.compile(loss='categorical_crossentropy', 
                optimizer = tf.keras.optimizers.Adam(lr=learning_rate),
                metrics='accuracy')

In [14]:
ensemble_model.fit(x_train, y_train, batch_size=batch_size,
         epochs=training_epochs)

Epoch 1/12
469/469 [==============================] - 34s 69ms/step - loss: 0.3363 - accuracy: 0.9247
Epoch 2/12
469/469 [==============================] - 33s 69ms/step - loss: 0.0382 - accuracy: 0.9905
Epoch 3/12
469/469 [==============================] - 35s 74ms/step - loss: 0.0253 - accuracy: 0.9933
Epoch 4/12
469/469 [==============================] - 34s 73ms/step - loss: 0.0182 - accuracy: 0.9954
Epoch 5/12
469/469 [==============================] - 34s 73ms/step - loss: 0.0131 - accuracy: 0.9969
Epoch 6/12
469/469 [==============================] - 34s 72ms/step - loss: 0.0099 - accuracy: 0.9977
Epoch 7/12
469/469 [==============================] - 34s 73ms/step - loss: 0.0083 - accuracy: 0.9981
Epoch 8/12
469/469 [==============================] - 34s 72ms/step - loss: 0.0067 - accuracy: 0.9985
Epoch 9/12
469/469 [==============================] - 34s 73ms/step - loss: 0.0060 - accuracy: 0.9986
Epoch 10/12
469/469 [==============================] - 34s 72ms/step - loss: 0.004

In [15]:
y_predicted = ensemble_model.predict(x_test)
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print('Index : ', random_index,
         '\tActual : ', np.argmax(np.array(y_test)[random_index]),
         '\tPredicted y : ', np.argmax(y_predicted[random_index]))

Index :  626 	Actual :  7 	Predicted y :  7
Index :  7695 	Actual :  7 	Predicted y :  7
Index :  2435 	Actual :  0 	Predicted y :  0
Index :  8631 	Actual :  7 	Predicted y :  7
Index :  2543 	Actual :  8 	Predicted y :  8
Index :  1726 	Actual :  0 	Predicted y :  0
Index :  4064 	Actual :  7 	Predicted y :  7
Index :  3171 	Actual :  5 	Predicted y :  5
Index :  2782 	Actual :  6 	Predicted y :  6
Index :  7704 	Actual :  4 	Predicted y :  4


In [16]:
evaluation = ensemble_model.evaluate(x_test, y_test)
print('Loss : ', round(evaluation[0], 5))
print('Accuracy : ', round(evaluation[1], 5))

313/313 [==============================] - 2s 4ms/step - loss: 0.0219 - accuracy: 0.9929
Loss :  0.02195
Accuracy :  0.9929
